Vadar bag of words Snetimental Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: change directory to current python notebooks on Google drive
import os
os.chdir('/content/drive/MyDrive/MRP/Dataset')

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import string


import matplotlib.pyplot as plt
import seaborn as sns

# stylesheet for plots
plt.style.use('ggplot')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

# Load Data

In [ ]:
# df = pd.read_excel('ugo_manual_raw.xlsx', index_col=0)
df = pd.read_csv('drugLibTrain_raw.tsv', sep='\t')
df = df.rename(columns={'Unnamed: 0': 'url_id'})
df = df.reset_index().rename(columns={'index': 'id'})
# side_effect_vader_score.rename(columns={"neg": "sd_neg"
df.head()

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


In [ ]:
# df = pd.read_excel('ugo_manual_raw.xlsx', index_col=0)
test_df = pd.read_csv('drugLibTest_raw.tsv', sep='\t')
test_df = test_df.rename(columns={'Unnamed: 0': 'url_id'})
test_df = test_df.reset_index().rename(columns={'index': 'id'})
# side_effect_vader_score.rename(columns={"neg": "sd_neg"
test_df.head()

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,0,1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...
1,1,3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...
2,2,3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...
3,3,969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...
4,4,696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...


Plot the counts of reviews

# Preprocessing

In [ ]:
print(bool(re.search(r"[a-zA-Z0-9]", '---')))
print(re.search(r"[a-zA-Z0-9]", 'a---'))
print(re.search(r"[a-zA-Z0-9]", 'a---o'))

False
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='a'>


In [ ]:
print(df['sideEffects'].unique())
print(df['effectiveness'].unique())

['Mild Side Effects' 'Severe Side Effects' 'No Side Effects'
 'Extremely Severe Side Effects' 'Moderate Side Effects']
['Highly Effective' 'Marginally Effective' 'Ineffective'
 'Considerably Effective' 'Moderately Effective']


In [ ]:
df.head()

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


Convert effectiveness and sideEffects to numerical values

In [ ]:

df['effectiveness_label'] = df['effectiveness'].map({'Highly Effective': 4, 'Considerably Effective': 3, 'Moderately Effective': 2, 'Marginally Effective': 1, 'Ineffective': 0})

df['sideEffects_label'] = df['sideEffects'].map({'No Side Effects': 4, 'Mild Side Effects': 3, 'Moderate Side Effects': 2,'Severe Side Effects': 1, 'Extremely Severe Side Effects': 0})

In [ ]:
df['rating'] = df['rating'] - 1

In [ ]:

test_df['effectiveness_label'] = test_df['effectiveness'].map({'Highly Effective': 4, 'Considerably Effective': 3, 'Moderately Effective': 2, 'Marginally Effective': 1, 'Ineffective': 0})

test_df['sideEffects_label'] = test_df['sideEffects'].map({'No Side Effects': 4, 'Mild Side Effects': 3, 'Moderate Side Effects': 2,'Severe Side Effects': 1, 'Extremely Severe Side Effects': 0})

In [ ]:
test_df['rating'] = test_df['rating'] - 1

In [ ]:
# Function to categorize ratings into sentiment labels
def label_sentiment(row):
    if row['rating'] >= 7:
      return "Positive"
    elif row['rating'] >= 4 and row['rating'] < 7:
      return "Neutral"
    elif row['rating'] >= 0 and row['rating'] < 4:
      return "Negative"
sentiment_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
df["label_sentiment_name"] = df.apply(label_sentiment, axis=1)
df["label_sentiment"] = df['label_sentiment_name'].map(sentiment_map)

test_df["label_sentiment_name"] = test_df.apply(label_sentiment, axis=1)
test_df["label_sentiment"] = test_df['label_sentiment_name'].map(sentiment_map)

In [ ]:
df.head()

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,effectiveness_label,sideEffects_label,label_sentiment_name,label_sentiment
0,0,2202,enalapril,3,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",4,3,Negative,0
1,1,3117,ortho-tri-cyclen,0,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest...",4,1,Negative,0
2,2,1146,ponstel,9,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,4,4,Positive,2
3,3,3947,prilosec,2,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,1,3,Negative,0
4,4,1951,lyrica,1,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above,1,1,Negative,0


Refill 'see above' with benefit reviews

In [ ]:
df[df['sideEffectsReview'] == 'see above']


,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,effectiveness_label,sideEffects_label,label_sentiment_name,label_sentiment
28,28,1959,lyrica,0,Marginally Effective,Severe Side Effects,"fibromyalgia, failed back syndrome","It seemed to work at first, but then the side ...",see above,Lyrica 150mg twice a day,1,1,Negative,0
1051,1051,1496,metrogel,3,Considerably Effective,Mild Side Effects,rosacea,I found metro gel to be irritating and drying....,see above,see above,3,3,Negative,0
1149,1149,819,ambien,2,Moderately Effective,Severe Side Effects,insomnia,"Fell fast asleep for the first few months, the...",see above,For menopausal insomnia and PTS from cancer,2,1,Negative,0


In [ ]:
test_df[test_df['sideEffectsReview'] == 'see above']

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,effectiveness_label,sideEffects_label,label_sentiment_name,label_sentiment
597,597,2002,vicodin,2,Moderately Effective,Moderate Side Effects,tendonitis pain,I took vicodin for pain in my wrist but i had ...,see above,This was prescribed by a physical therapist. I...,2,2,Negative,0
653,653,3605,retin-a,5,Moderately Effective,Moderate Side Effects,acne,I had some experience with Retin-A when I was ...,see above,see above,2,2,Neutral,1


In [ ]:
df.loc[df['sideEffectsReview'] == 'see above', 'sideEffectsReview'] = df[df['sideEffectsReview'] == 'see above']['benefitsReview']

In [ ]:
test_df.loc[test_df['sideEffectsReview'] == 'see above', 'sideEffectsReview'] = test_df[test_df['sideEffectsReview'] == 'see above']['benefitsReview']

In [ ]:
def preprocess(review):
  # print(review)
  if review == np.nan or review == 'nan' \
    or review == 'None.' or review == 'none' \
    or review == 'no' or review == '*' \
    or review == '..' or ((isinstance(review, int) or isinstance(review, float)) and np.isnan(review)):
    review = 'None'

  if not bool(re.search(r"[a-zA-Z0-9]", review)):
    print(review)
    review = 'None'

  review = review.lower()
  return review


df['sideEffectsReview'] = df['sideEffectsReview'].apply(preprocess)
df['benefitsReview'] = df['benefitsReview'].apply(preprocess)

---


In [ ]:
test_df['sideEffectsReview'] = test_df['sideEffectsReview'].apply(preprocess)
test_df['benefitsReview'] = test_df['benefitsReview'].apply(preprocess)

# Fine-tuning BioBERT

Load modules

Used Hugging Face Trainer API for training

Model predicts negative, neutral, or positive sentiment

evaluating accuracy

improving performance

To do

✅ Fine-tuned BioBERT on side effect reviews

✅ Used Hugging Face Trainer API for training

✅ Model predicts negative, neutral, or positive sentiment

evaluating accuracy

improving performance

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [ ]:
import torch

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

**SIDE EFFECTS DATA**

For side effect, use BiobERT.

For side sideEffectsReview and it's label sideEffects

In [ ]:
side_effect_df = df[['sideEffectsReview', 'sideEffects', 'sideEffects_label']]
side_effect_df.head()
benefits_df = df[['benefitsReview', 'effectiveness', 'effectiveness_label']]
# side_effect_df['label']
benefits_df.head()

,benefitsReview,effectiveness,effectiveness_label
0,slowed the progression of left ventricular dys...,Highly Effective,4
1,although this type of birth control has more c...,Highly Effective,4
2,i was used to having cramps so badly that they...,Highly Effective,4
3,the acid reflux went away for a few months aft...,Marginally Effective,1
4,i think that the lyrica was starting to help w...,Marginally Effective,1


In [ ]:
df.head()

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,effectiveness_label,sideEffects_label,label_sentiment_name,label_sentiment
0,0,2202,enalapril,3,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",4,3,Negative,0
1,1,3117,ortho-tri-cyclen,0,Highly Effective,Severe Side Effects,birth prevention,although this type of birth control has more c...,"heavy cycle, cramps, hot flashes, fatigue, lon...","I Hate This Birth Control, I Would Not Suggest...",4,1,Negative,0
2,2,1146,ponstel,9,Highly Effective,No Side Effects,menstrual cramps,i was used to having cramps so badly that they...,heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,4,4,Positive,2
3,3,3947,prilosec,2,Marginally Effective,Mild Side Effects,acid reflux,the acid reflux went away for a few months aft...,"constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,1,3,Negative,0
4,4,1951,lyrica,1,Marginally Effective,Severe Side Effects,fibromyalgia,i think that the lyrica was starting to help w...,i felt extremely drugged and dopey. could not...,See above,1,1,Negative,0


In [ ]:
def combine_texts(row):
    benefit = str(row["benefitsReview"]) if pd.notna(row["benefitsReview"]) else ""
    side_effect = str(row["sideEffectsReview"]) if pd.notna(row["sideEffectsReview"]) else ""

    # Combine both with a [SEP] token
    return benefit + " [SEP] " + side_effect



df["combined_text"] = df.apply(combine_texts, axis=1)
df["label_sentiment_name"] = df.apply(label_sentiment, axis=1)

In [ ]:

test_df["combined_text"] = test_df.apply(combine_texts, axis=1)
test_df["label_sentiment_name"] = test_df.apply(label_sentiment, axis=1)

In [ ]:
df.head()

,id,url_id,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,effectiveness_label,sideEffects_label,combined_text,label_sentiment_name,label_sentiment
0,0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",4,3,slowed the progression of left ventricular dys...,Neutral,1
1,1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,although this type of birth control has more c...,"heavy cycle, cramps, hot flashes, fatigue, lon...","I Hate This Birth Control, I Would Not Suggest...",4,1,although this type of birth control has more c...,Negative,0
2,2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,i was used to having cramps so badly that they...,heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,4,4,i was used to having cramps so badly that they...,Positive,2
3,3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,the acid reflux went away for a few months aft...,"constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,1,3,the acid reflux went away for a few months aft...,Negative,0
4,4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,i think that the lyrica was starting to help w...,i felt extremely drugged and dopey. could not...,See above,1,1,i think that the lyrica was starting to help w...,Negative,0


In [ ]:
train_combined_df = df[['combined_text', 'label_sentiment_name','label_sentiment']]
train_combined_df.head()

,combined_text,label_sentiment_name,label_sentiment
0,slowed the progression of left ventricular dys...,Negative,0
1,although this type of birth control has more c...,Negative,0
2,i was used to having cramps so badly that they...,Positive,2
3,the acid reflux went away for a few months aft...,Negative,0
4,i think that the lyrica was starting to help w...,Negative,0


In [ ]:
test_combined_df = test_df[['combined_text', 'label_sentiment_name','label_sentiment']]
test_combined_df.head()

,combined_text,label_sentiment_name,label_sentiment
0,the antibiotic may have destroyed bacteria cau...,Positive,2
1,lamictal stabilized my serious mood swings. on...,Positive,2
2,initial benefits were comparable to the brand ...,Negative,0
3,it controlls my mood swings. it helps me think...,Positive,2
4,within one week of treatment superficial acne ...,Positive,2


In [ ]:
# combined_df['label_sentiment'].value_counts()

,count
label_sentiment,
2,2130
0,554
1,423


In [ ]:
# model_name = 'dmis-lab/biobert-base-cased-v1.1'
model_name = 'dmis-lab/biobert-v1.1'
bioBert_tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
label_names = train_combined_df['label_sentiment_name'].unique()
labels = train_combined_df['label_sentiment'].unique()


NUM_LABELS = len(labels)
labels
id2label = {i:str(l) for i,l in enumerate(label_names)}
label2id = {str(l):i for i,l in enumerate(labels)}
id2label

{0: 'Negative', 1: 'Positive', 2: 'Neutral'}

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = NUM_LABELS) #id2label=id2label, label2id=label2id)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Train, validation split

In [ ]:
# size = (3 * train_combined_df.shape[0])//4
train_data = list(train_combined_df['combined_text'])
# val_data = list(train_combined_df['combined_text'][size:])

train_label = list(train_combined_df['label_sentiment'])
# val_label = list(train_combined_df['label_sentiment'][size:])

In [ ]:
# test_combined_df
test_data = list(test_combined_df['combined_text'])
# val_data = list(train_combined_df['combined_text'][size:])

test_label = list(test_combined_df['label_sentiment'])

Tokenize the data

In [ ]:
train_encoding = bioBert_tokenizer(train_data, truncation=True, padding=True)
test_encoding = bioBert_tokenizer(test_data, truncation=True, padding=True)
# val_encoding = bioBert_tokenizer(val_data, truncation=True, padding=True)

In [ ]:

from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        # print(idx)
        # print(len(self.labels))
        # item = {key: torch.tensor(val[idx]) if idx < len(self.labels) else -1 for key, val in self.encodings.items()}
        item = {}
        for key, val in self.encodings.items():

          try:
            item[key] = torch.tensor(val[idx])
          except IndexError:
            print(idx)
            print('key:' , key)
            print('val: ', len(val))
            item[key] = torch.tensor(-1)
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = MyDataset(train_encoding, train_label)
test_dataset = MyDataset(test_encoding, test_label)

Train with Trainer class

In [ ]:
training_args = TrainingArguments(
    output_dir='./COMBINEDBIOBERTMODEL',
    do_train=True,
    # do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',
    logging_steps=50,
    # eval_strategy='steps',
    # eval_steps=50,
    save_strategy='steps',
    fp16=True
    # load_best_model_at_end=True
)

Compute matrics
attempt with Macro-averaging. Macro Averaging computes the average score of Precision, recall, and F1 class. It gives equal weights to each class

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  Precision, Recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  return {
      'Accuracy': acc,
      'F1': f1,
      'Precision': Precision,
      'Recall': Recall
  }


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=train_dataset,
    train_dataset=train_dataset
    # eval_dataset=val_dataset,
    # compute_metrics=compute_metrics
)

In [ ]:
results=trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: uokehi (uokehi-toronto-metropolitan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,1.047900
100,0.938400
150,0.797200
200,0.774700
250,0.620500
300,0.627900
350,0.631200
400,0.518900
450,0.362100
500,0.338800


In [ ]:
pred_logit = trainer.predict(test_dataset)

In [ ]:
pred_logit

PredictionOutput(predictions=array([[ 1.6064453 ,  0.3334961 , -2.2089844 ],
       [-2.1816406 ,  0.5551758 ,  2.2949219 ],
       [-1.8613281 ,  0.45996094,  2.1269531 ],
       ...,
       [ 0.77734375,  0.60009766, -1.5556641 ],
       [-2.2265625 , -0.703125  ,  3.4960938 ],
       [-1.1953125 ,  1.1210938 , -0.16809082]], dtype=float32), label_ids=array([2, 2, 0, ..., 2, 2, 2]), metrics={'test_loss': 0.8153778314590454, 'test_runtime': 1.3742, 'test_samples_per_second': 753.914, 'test_steps_per_second': 24.015})

In [ ]:
len(pred_logit.label_ids)

1036

In [ ]:
test_combined_df['predicted_label'] = pred_logit.label_ids
# .loc[row_indexer,col_indexer]

<ipython-input-50-9e0ae7c4fa61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_combined_df['predicted_label'] = pred_logit.label_ids


In [ ]:
test_combined_df.head()

,combined_text,label_sentiment_name,label_sentiment,predicted_label
0,the antibiotic may have destroyed bacteria cau...,Positive,2,2
1,lamictal stabilized my serious mood swings. on...,Positive,2,2
2,initial benefits were comparable to the brand ...,Negative,0,0
3,it controlls my mood swings. it helps me think...,Positive,2,2
4,within one week of treatment superficial acne ...,Positive,2,2


In [ ]:
test_combined_df.to_excel('biobert_predicted_labels.xlsx')

In [ ]:
compute_metrics(pred_logit)

{'Accuracy': 0.6978764478764479,
 'F1': 0.6521173224896758,
 'Precision': 0.6643640206049398,
 'Recall': 0.6444244186450209}

In [ ]:
pd.DataFrame.from_dict(compute_metrics(pred_logit), orient='index', columns=['Value']).to_excel('biobert_metrics.xlsx')

In [ ]:
training_args1 = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./benefitmodel1',
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=500,
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    eval_steps=100,
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer1 = Trainer(
    model=model,
    args=training_args1,
    # train_dataset=train_dataset,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_metrics=compute_metrics
)

In [ ]:
results1=trainer1.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.212400,1.313975,0.657658,0.579355,0.658046,0.567157
2,0.167700,1.515687,0.656371,0.570201,0.643053,0.559222
3,0.191800,1.876794,0.584299,0.506804,0.587446,0.511950
4,0.273200,1.478339,0.634492,0.568335,0.605374,0.556795
5,0.125400,1.845688,0.643501,0.564694,0.606289,0.550176


In [ ]:
df['label'].value_counts()

In [ ]:
# from transformers import get_scheduler

# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=100,
#     num_training_steps=total_steps
# )
# Add More Layers (Increase Model Capacity)

# import torch.nn as nn

# class BioBERTClassifier(nn.Module):
#     def __init__(self, model_name="dmis-lab/biobert-base-cased-v1.1", num_labels=3):
#         super().__init__()
#         self.bert = AutoModel.from_pretrained(model_name)
#         self.fc = nn.Sequential(
#             nn.Linear(768, 512),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(512, num_labels)
#         )

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         return self.fc(outputs.pooler_output)

In [ ]:
# Building a Transformer model from scratch requires implementing self-attention, positional encoding, and multi-layer encoders/decoders. Below is a step-by-step guide to building a Transformer model using PyTorch.

